# <할수있다 퀀트투자> 프로젝트
### 전략 : 강환국 슈퍼 가치 전략
### 작성자 : 정경륜
<br>

## 전략소개
> <br>

> ### 매수전략 
> 1. 한국 시가총액 하위 20% 주식을 대상으로 PBR, PCR, PER, PSR 각각 순위를 매김
> 2. 네 개 지표의 순위를 더해 통합 순위 작성
> 3. 통합 순위가 높은 종목 50개 매수
> ### 매도전략
> * 연 1회 리벨런싱
> ### 기대 CAGR 
> * 25% 이상  
> <br>

## 용어 설명
<br>

* **PER**(*Price Earning Ratio*) = $주가 \over 주당순이익(EPS)$ 
        : 한 기업이 얻은 순이익 1원을 증권시장이 얼마의 가격으로 평가하고 있는가를 나타내는 수치다.

<br>

* **PCR**(*Price Cash Flow Ratio*) = $주가 \over 주당영업현금흐름(CFPS)$ 
        : 특정 기업이 얻은 영업현금흐름 1원을 증권시장이 얼마의 가격으로 평가하고 있는지 나타내는 수치다.

<br>

* **PSR**(*Price Selling Ratio*) = $주가 \over 주당매출액(SPS)$ 
         : 기업의 매출액 1원을 증권시장이 얼마의 가격으로 평가하는가를 나타내는 수치다.
         
<br>

* **PBR**(*Price Book Ratio*) = $주가 \over 주당순자산(BPS)$ 
        : 기업의 순자산 1원을 증권시장이 얼마의 가격으로 평가하는가를 나타내는 수치다.
<br>

위의 지표들을 구하기 위해 필요한 지표들 입니다. (아래 지표들의 공식은 fnguide.com에서 발췌해왔습니다.)

<br>

* **EPS**(*Earning Per Share*) = $지배주주순수익 \over 수정평균주식수$ 

<br>

* **CFPS**(*Cash Flow Per Share*) = $현금흐름 \over 수정평균주식수$ 

<br>

* **SPS**(*Sales Per Share*) = $영업수익(매출액)\over 수정평균주식수$ 

<br>

* **BPS**(*Book-value Per Share*) = $지배주주순자산 \over 수정기말주식수$ 

<br>

## 1. 시가총액 하위 20%의 소형주 고르기

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
from pykrx import stock
import datetime as dt
import time

#오늘 날짜 구하기
dt_now = str(dt.datetime.now().date())
print(f'{dt_now} 기준')
dt_now = ''.join(c for c in dt_now if c not in '-')

# 시가총액이 적은 순으로 나열
market_cap_df = stock.get_market_cap_by_ticker(dt_now)
market_cap_df = market_cap_df[['시가총액']]
market_cap_df = market_cap_df.sort_values(by=['시가총액'], ascending=True)
market_cap_df

2021-09-24 기준


,시가총액
티커,
225860,1277785350
225850,1414575217
267810,2266355000
240340,3342976677
001529,3418408200
...,...
005935,59000976390000
207940,61004130000000
035420,66608806672500


In [2]:
length = int(len(market_cap_df.index) * 0.2)
print(f'시가총액 하위 20%의 종목 개수 : {length}')

# 시가총액 하위 20% 소형주 선별
low_market_cap_df = market_cap_df.iloc[:length]
ticker_list = np.array(low_market_cap_df.index)
ticker_list


시가총액 하위 20%의 종목 개수 : 516


array(['225860', '225850', '267810', '240340', '001529', '215050',
       '245450', '276240', '093510', '140660', '270210', '114920',
       '000547', '329020', '002787', '318660', '032685', '001527',
       '116100', '288490', '001525', '009275', '002785', '001067',
       '014915', '016385', '242350', '001065', '308700', '179720',
       '039230', '270020', '111820', '021045', '217910', '347140',
       '00781K', '323210', '267060', '335870', '114570', '121890',
       '000545', '140290', '373340', '323280', '388790', '058420',
       '000227', '349720', '011155', '014285', '000145', '373200',
       '329560', '331520', '121060', '004545', '285770', '185190',
       '343510', '217320', '000325', '005745', '004565', '353070',
       '330990', '266870', '344050', '006345', '337450', '08537M',
       '332710', '372290', '002995', '271850', '366330', '310870',
       '014825', '090355', '341160', '367360', '183410', '313750',
       '058220', '084695', '365590', '004415', '012205', '3530

## 2. 선정된 종목의 PER, PCR, PSR, PBR 지표 구하기 
### 1) 크롤링 테스트
PER, PCR, PSR, PBR을 구하기 위해 삼성전자 한 종목으로 크롤링 테스트를 진행합니다.

가장 최신의 지표를 구하기 위해 pykrx로부터 가장 최신의 수정종가를 불러온 후 이를 EPS, CFPS, SPS, BPS로 나눠 PER, PCR, PSR, PBR을 구하려 합니다.

크롤링 테스트에선 삼성전자의 가장 최신 분기의 EPS, CFPS, SPS, BPS를 불러오는 것까지만 하겠습니다.

In [3]:
import requests

# 삼성전자 단일 종목으로 크롤링 테스트
code = "005930"
url = f"http://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701"
header = {"User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"}
res = requests.get(url, headers=header)
res.raise_for_status()

df = pd.read_html(res.text)[1]
df

,IFRS 연결,2017/12,2018/12,2019/12,2020/12,2021/06
0,Per Share,Per Share,Per Share,Per Share,Per Share,Per Share
1,EPS계산에 참여한 계정 펼치기(원),5421,6024,3166,3841,2435
2,EBITDAPS계산에 참여한 계정 펼치기(원),9934,11717,8445,9765,5562
3,CFPS계산에 참여한 계정 펼치기(원),8321,9659,7523,8307,4766
4,SPS계산에 참여한 계정 펼치기(원),31414,33458,33919,34862,19000
5,BPS계산에 참여한 계정 펼치기(원),28971,35342,37528,39406,40361
6,Dividends,Dividends,Dividends,Dividends,Dividends,Dividends
7,"DPS(보통주,현금)(원)계산에 참여한 계정 펼치기",850,1416,1416,2994,722
8,"DPS(1우선주,현금)(원)계산에 참여한 계정 펼치기",851,1417,1417,2995,722
9,배당성향(현금)(%)계산에 참여한 계정 펼치기,14.09,21.92,44.73,77.95,NaN


In [4]:
df = df.drop(2)
df = df.iloc[1:5]
value = df['2021/06'].values

df = pd.DataFrame(columns=['EPS', 'CFPS', 'SPS', 'BPS'])
df.loc[code] = value
df

,EPS,CFPS,SPS,BPS
005930,2435,4766,19000,40361


크롤링이 잘 진행된 것을 확인할 수 있습니다.

### 2) 크롤링 진행

실제 크롤링을 진행합니다.

앞서 크롤링 테스트를 토대로 dataframe을 알맞게 수정하는 함수를 df_modify()로 정의하였습니다.

In [5]:
fundamental_df = pd.DataFrame(columns=['EPS', 'CFPS', 'SPS', 'BPS'])

def df_modify(df, date, ticker):
    df = df.drop(2)
    df = df.iloc[1:5]
    fundamental_df.loc[ticker] = df[date].values
    
    return fundamental_df

In [6]:
date = '2021/06'
length = len(ticker_list)
p = 0

for ticker in ticker_list:
    p += 1
    try:
        url = f"http://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{ticker}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701"
        header = {"User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"}
        res = requests.get(url, headers=header)
        res.raise_for_status()

        df = pd.read_html(res.text)[1]
        print(f'{ticker} : Success! ({p}/{length})')
        fundamental_df = df_modify(df, date, ticker)
    except:
        print(f'{ticker} : No tables found ({p}/{length})')

fundamental_df 

225860 : Success! (1/516)
225850 : Success! (2/516)
267810 : Success! (3/516)
240340 : Success! (4/516)
001529 : No tables found (5/516)
215050 : Success! (6/516)
245450 : Success! (7/516)
276240 : Success! (8/516)
093510 : Success! (9/516)
140660 : Success! (10/516)
270210 : Success! (11/516)
114920 : Success! (12/516)
000547 : No tables found (13/516)
329020 : Success! (14/516)
002787 : No tables found (15/516)
318660 : Success! (16/516)
032685 : No tables found (17/516)
001527 : No tables found (18/516)
116100 : Success! (19/516)
288490 : Success! (20/516)
001525 : No tables found (21/516)
009275 : No tables found (22/516)
002785 : No tables found (23/516)
001067 : No tables found (24/516)
014915 : No tables found (25/516)
016385 : No tables found (26/516)
242350 : Success! (27/516)
001065 : No tables found (28/516)
308700 : Success! (29/516)
179720 : Success! (30/516)
039230 : Success! (31/516)
270020 : Success! (32/516)
111820 : Success! (33/516)
021045 : No tables found (34/516)


,EPS,CFPS,SPS,BPS
225860,NaN,NaN,NaN,NaN
225850,NaN,NaN,NaN,NaN
267810,NaN,NaN,NaN,NaN
240340,NaN,NaN,NaN,NaN
215050,NaN,NaN,NaN,NaN
...,...,...,...,...
221980,1104,1188,6426,21768
109070,17,74,1522,628
049120,-220,-130,439,1438
062860,-126,-69,1750,7875


Table이 없는 경우는 우선주 종목 입니다. 

### 3) EPS, CFPS, SPS, BPS 테이블 분석

In [7]:
fundamental_df.isnull().sum()

EPS     111
CFPS    107
SPS     163
BPS     163
dtype: int64

우선 null 값이 있는 항들은 지표를 계산할 수 없으니 모두 제거하였습니다.

In [8]:
fundamental_df = fundamental_df.dropna(axis=0)
fundamental_df

,EPS,CFPS,SPS,BPS
039230,-58,-44,429,1191
111820,-41,-34,356,500
114570,-58,-58,137,4139
121890,134,152,1222,782
058420,-882,-765,1792,488
...,...,...,...,...
221980,1104,1188,6426,21768
109070,17,74,1522,628
049120,-220,-130,439,1438
062860,-126,-69,1750,7875


In [9]:
fundamental_df.isnull().sum()

EPS     0
CFPS    0
SPS     0
BPS     0
dtype: int64

In [10]:
fundamental_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 270 entries, 039230 to 128540
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   EPS     270 non-null    object
 1   CFPS    270 non-null    object
 2   SPS     270 non-null    object
 3   BPS     270 non-null    object
dtypes: object(4)
memory usage: 10.5+ KB


In [11]:
fundamental_df = fundamental_df.apply(pd.to_numeric)
fundamental_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 270 entries, 039230 to 128540
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   EPS     270 non-null    int64
 1   CFPS    270 non-null    int64
 2   SPS     270 non-null    int64
 3   BPS     270 non-null    int64
dtypes: int64(4)
memory usage: 10.5+ KB


### 4) PER, PCR, PSR, PBR 계산

수정주가를 가져오기 위해서 pykrx의 get_market_ohlcv_by_date() 함수를 이용하려 합니다. 수정주가를 얻기 위해선 adjust 옵션을 True로 설정해주면 되지만 default 값이 True이기에 그냥 진행해도 무방합니다. 다만 여기선 수정주가를 사용함을 명시하기 위해 옵션을 표시하겠습니다.

In [12]:
ticker_list = fundamental_df.index
ticker_list

Index(['039230', '111820', '114570', '121890', '058420', '058220', '101680',
       '033790', '050540', '038340',
       ...
       '006200', '227100', '063760', '050760', '007610', '221980', '109070',
       '049120', '062860', '128540'],
      dtype='object', length=270)

현재일로부터 가장 가까운 영업일과 해당 영업일 전날을 구합니다.

In [14]:
target_date = stock.get_nearest_business_day_in_a_week(date=dt_now)
date_formatter = "%Y%m%d"
today = dt.datetime.strptime(target_date, date_formatter)
previous_date = today - dt.timedelta(1)
previous_date = dt.datetime.strftime(previous_date, date_formatter)
print(f'target date : {target_date} ')
print(f'previous date : {previous_date}')

target date : 20210924 
previous date : 20210923


In [15]:
# 현재일로부터 가장 가까운 영업일을 구함

value_df = pd.DataFrame(columns=['PER', 'PCR', 'PSR', 'PBR'])

length = len(fundamental_df.index)
p = 0

for ticker in ticker_list:
    df = stock.get_market_ohlcv_by_date(previous_date, target_date, ticker, adjusted=True)
    adjusted_price = df['종가']     # 수정종가
    value = adjusted_price[0] / fundamental_df.loc[ticker]  # 지표 계산
    value_df.loc[ticker] = value.values
    p += 1
    print(f'Processing : {p}/{length}')
    time.sleep(1)

value_df

Processing : 1/270
Processing : 2/270
Processing : 3/270
Processing : 4/270
Processing : 5/270
Processing : 6/270
Processing : 7/270
Processing : 8/270
Processing : 9/270
Processing : 10/270
Processing : 11/270
Processing : 12/270
Processing : 13/270
Processing : 14/270
Processing : 15/270
Processing : 16/270
Processing : 17/270
Processing : 18/270
Processing : 19/270
Processing : 20/270
Processing : 21/270
Processing : 22/270
Processing : 23/270
Processing : 24/270
Processing : 25/270
Processing : 26/270
Processing : 27/270
Processing : 28/270
Processing : 29/270
Processing : 30/270
Processing : 31/270
Processing : 32/270
Processing : 33/270
Processing : 34/270
Processing : 35/270
Processing : 36/270
Processing : 37/270
Processing : 38/270
Processing : 39/270
Processing : 40/270
Processing : 41/270
Processing : 42/270
Processing : 43/270
Processing : 44/270
Processing : 45/270
Processing : 46/270
Processing : 47/270
Processing : 48/270
Processing : 49/270
Processing : 50/270
Processin

,PER,PCR,PSR,PBR
039230,-3.000000,-3.954545,0.405594,0.146096
111820,-5.365854,-6.470588,0.617978,0.440000
114570,-10.689655,-10.689655,4.525547,0.149795
121890,5.514925,4.861842,0.604746,0.945013
058420,-2.448980,-2.823529,1.205357,4.426230
...,...,...,...,...
221980,14.855072,13.804714,2.552132,0.753399
109070,241.176471,55.405405,2.693824,6.528662
049120,-11.431818,-19.346154,5.728929,1.748957
062860,-53.412698,-97.536232,3.845714,0.854603


### 5) 제외조건 설정

PBR이 터무니없이 낮은 경우엔 기업이 망하기 일보 직전일 확률이 높다고 합니다. 따라서 PBR이 0.2 이하인 기업들은 제외하였습니다. 

또한 나머지 지표들이 0 또는 음수가 나온 경우엔 적자를 보고 있는 기업이기에 이 또한 제외하였습니다.

In [16]:
del_index = value_df[(value_df['PBR'] <= 0.2) | (value_df['PER'] <= 0) | (value_df['PSR'] <= 0) | (value_df['PCR'] <= 0)].index
value_df = value_df.drop(del_index)

## 3. 네 개의 지표 순위와 순위 합 구하기
### 1) 10분위로 rank 매기기

In [17]:
value_df.describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9,1])

/opt/homebrew/Caskroom/miniforge/base/envs/quant/lib/python3.9/site-packages/numpy/lib/function_base.py:3961: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,PER,PCR,PSR,PBR
count,141.000000,141.000000,141.000000,141.000000
mean,71.240485,23.404427,inf,1.328588
std,181.546784,24.987284,NaN,0.921327
min,1.032212,1.019810,0.066687,0.237384
10%,7.146552,5.202559,0.530168,0.597920
20%,13.463415,9.682948,0.869118,0.729703
30%,17.663043,11.621053,1.148686,0.812041
40%,23.620690,13.846154,1.475818,0.945013
50%,28.437500,16.511111,1.891094,1.080858
60%,34.355828,20.326633,2.347670,1.253102


In [18]:
def change_to_rank(df, col, val):
    if val <= df[col].quantile(.1):
        return 10
    elif val <= df[col].quantile(.2):
        return 9
    elif val <= df[col].quantile(.3):
        return 8
    elif val <= df[col].quantile(.4):
        return 7
    elif val <= df[col].quantile(.5):
        return 6
    elif val <= df[col].quantile(.6):
        return 5
    elif val <= df[col].quantile(.7):
        return 4
    elif val <= df[col].quantile(.8):
        return 3
    elif val <= df[col].quantile(.9):
        return 2
    else :
        return 1


In [19]:
rank_df = pd.DataFrame(columns=['PER', 'PCR', 'PSR', 'PBR'])

for col in value_df.columns:
    rank_df[col] = value_df[col].apply(lambda x : change_to_rank(value_df, col, x))
rank_df

,PER,PCR,PSR,PBR
121890,10,10,9,7
033790,10,9,5,9
050540,1,9,10,1
141020,10,10,1,10
178600,9,10,10,6
...,...,...,...,...
006200,10,10,10,10
050760,8,8,7,6
221980,8,7,4,8
109070,1,1,4,1


### 2) Rank 합 계산

In [20]:
rank_df['rank_sum'] = rank_df.sum(axis=1)
rank_df = rank_df.sort_values('rank_sum', ascending=False)
rank_df

,PER,PCR,PSR,PBR,rank_sum
024830,10,10,10,10,40
006200,10,10,10,10,40
019180,10,10,10,8,38
025530,9,9,9,10,37
900340,10,9,8,10,37
...,...,...,...,...,...
275630,1,1,1,4,7
109070,1,1,4,1,7
050320,3,1,1,1,6
356890,2,1,2,1,6


## 4. 통합순위 상위 50개 종목 선택

In [21]:
num = 50
selected_stock_df = value_df.loc[rank_df.iloc[:50].index]
selected_stock_df

,PER,PCR,PSR,PBR
024830,3.678969,1.946683,0.486746,0.237384
006200,6.853659,4.561688,0.429532,0.557319
019180,3.696099,2.982601,0.320114,0.761421
025530,13.219178,6.917563,0.743595,0.364014
900340,7.072539,5.783898,0.906977,0.475444
000760,1.032212,1.019810,1.840880,0.332785
121890,5.514925,4.861842,0.604746,0.945013
212560,13.011272,5.504087,0.874743,0.729703
091340,19.545455,6.833741,0.442948,0.693376
018500,5.941176,2.337963,0.470424,1.253102


In [22]:
selected_ticker_list = selected_stock_df.index
selected_stocks = {}
for ticker in selected_ticker_list:
    종목 = stock.get_market_ticker_name(ticker)
    selected_stocks[ticker] = 종목
    print(종목)

세원물산
한국전자홀딩스
티에이치엔
SJM홀딩스
윙입푸드
이화산업
에스디시스템
네오오토
S&K폴리텍
동원금속
대동고려삼
지에스이
신송홀딩스
ES큐브
아세아텍
연이비앤티
서진오토모티브
스타플렉스
세보엠이씨
미래산업
제이에스티나
스카이문스테크놀로지
패션플랫폼
대동기어
대동금속
한성기업
화천기계
동일철강
포티스
SHD
동원수산
코다코
제일테크노스
삼진
에이치케이
제룡산업
글로벌에스엠
에스폴리텍
백금T&A
동일기연
한창산업
제이엠티
SGA
우리로
케이씨피드
한일화학
케이디켐
유엔젤
서울리거
빛샘전자


강환국 슈퍼 가치 전략에 따르면 매수 전략에 따라 50개의 종목을 선별한 뒤 나와 정말 맞지 않은 종목을 제외하고 20-30개의 종목을 사는 것을 추천하고 있습니다. 따라서 아래의 50개의 종목에서 부채, 적자, 산업군 등을 분석하고 20-30개의 종목들을 고르면 됩니다. 

In [23]:
selected_stocks

{'024830': '세원물산',
 '006200': '한국전자홀딩스',
 '019180': '티에이치엔',
 '025530': 'SJM홀딩스',
 '900340': '윙입푸드',
 '000760': '이화산업',
 '121890': '에스디시스템',
 '212560': '네오오토',
 '091340': 'S&K폴리텍',
 '018500': '동원금속',
 '178600': '대동고려삼',
 '053050': '지에스이',
 '006880': '신송홀딩스',
 '050120': 'ES큐브',
 '050860': '아세아텍',
 '090740': '연이비앤티',
 '122690': '서진오토모티브',
 '115570': '스타플렉스',
 '011560': '세보엠이씨',
 '025560': '미래산업',
 '026040': '제이에스티나',
 '033790': '스카이문스테크놀로지',
 '225590': '패션플랫폼',
 '008830': '대동기어',
 '020400': '대동금속',
 '003680': '한성기업',
 '010660': '화천기계',
 '023790': '동일철강',
 '141020': '포티스',
 '001770': 'SHD',
 '030720': '동원수산',
 '046070': '코다코',
 '038010': '제일테크노스',
 '032750': '삼진',
 '044780': '에이치케이',
 '147830': '제룡산업',
 '900070': '글로벌에스엠',
 '050760': '에스폴리텍',
 '046310': '백금T&A',
 '032960': '동일기연',
 '079170': '한창산업',
 '094970': '제이엠티',
 '049470': 'SGA',
 '046970': '우리로',
 '025880': '케이씨피드',
 '007770': '한일화학',
 '221980': '케이디켐',
 '072130': '유엔젤',
 '043710': '서울리거',
 '072950': '빛샘전자'}